In [ ]:
#@title *code:* library imports
import tensorflow.keras.layers as tfkl
import tensorflow.keras as tfk
%tensorflow_version 2.x
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import dill
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/course-IA2/Work-Max/Proyecto')
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/course-IA2/Work-Max/Proyecto


In [ ]:
class lstm_bottleneck(tfkl.Layer):
    def __init__(self, lstm_units, time_steps, **kwargs):
        self.lstm_units = lstm_units
        self.time_steps = time_steps
        self.lstm_layer = tfkl.Bidirectional(tfkl.LSTM(lstm_units,return_sequences=False))
        self.repeat_layer = tfkl.RepeatVector(time_steps)
        super(lstm_bottleneck, self).__init__(**kwargs)
    
    def call(self, inputs):
        # just call the two initialized layers
        return self.repeat_layer(self.lstm_layer(inputs))
    
    def compute_mask(self, inputs, mask=None):
        # return the input_mask directly
        return mask


In [ ]:
latent_dim = 7

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tfk.Sequential()
    self.encoder.add(tfkl.Input(shape=(32, 3)))
    self.encoder.add(tfk.layers.Masking(mask_value=0.))

    self.encoder.add(tfkl.Bidirectional(tfkl.LSTM(units=32,return_sequences=True)))
    self.encoder.add(lstm_bottleneck(lstm_units=self.latent_dim, time_steps=32))

    self.decoder = tfk.Sequential()

    self.decoder.add(tfkl.Bidirectional(tfkl.LSTM(units=32,return_sequences=True)))
    self.decoder.add(tfkl.Bidirectional(tfkl.LSTM(units=3,return_sequences=True)))
    self.decoder.add(tfk.layers.Dense(3,activation='elu'))
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  

autoencoder = Autoencoder(latent_dim,)

In [ ]:
optimizer=tfk.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)
autoencoder.compile(optimizer=optimizer, loss=losses.MeanSquaredError(),metrics=['mse',tfk.metrics.RootMeanSquaredError()])
autoencoder.build((None,32,3))
print(autoencoder.summary())

Model: "autoencoder_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_4 (Sequential)    (None, 32, 14)            13248     
_________________________________________________________________
sequential_5 (Sequential)    (None, 32, 3)             13685     
Total params: 26,933
Trainable params: 26,933
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 32, 3)             0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 32, 64)            9216      
_________________________________________________________________
lstm_bottleneck_2 (lstm_bott (None, 32, 14)            4032      
Total params: 13,248
Trainable params: 13,248
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (None, 32, 64)            12032     
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 32, 6)             1632      
____________________________

In [ ]:
ogle_train_data = np.load('data/ogle_train_data.npy')
ogle_train_data_lengths = np.load('data/ogle_train_data_lengths.npy')

In [ ]:
split=0.8
n=int(split*len(ogle_train_data))
x_train=ogle_train_data[:n]
x_test=ogle_train_data[n:]
np.shape(x_test)

(33215, 32, 3)

In [ ]:
history=autoencoder.fit(x_train, x_train,
                epochs=50,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50
4152/4152 [==============================] - 153s 37ms/step - loss: 6723.3174 - mse: 8818.4424 - root_mean_squared_error: 93.9067 - val_loss: 6684.2524 - val_mse: 8756.1846 - val_root_mean_squared_error: 93.5745
Epoch 2/50
4152/4152 [==============================] - 145s 35ms/step - loss: 6629.6064 - mse: 8695.5293 - root_mean_squared_error: 93.2497 - val_loss: 6594.5205 - val_mse: 8638.6387 - val_root_mean_squared_error: 92.9443
Epoch 3/50
4152/4152 [==============================] - 145s 35ms/step - loss: 6544.0586 - mse: 8583.3223 - root_mean_squared_error: 92.6461 - val_loss: 6512.9214 - val_mse: 8531.7461 - val_root_mean_squared_error: 92.3674
Epoch 4/50
4152/4152 [==============================] - 143s 35ms/step - loss: 6465.6494 - mse: 8480.4795 - root_mean_squared_error: 92.0896 - val_loss: 6437.8735 - val_mse: 8433.4355 - val_root_mean_squared_error: 91.8337
Epoch 5/50
4152/4152 [==============================] - 143s 34ms/step - loss: 6395.1016 - mse: 8387.9473 - 